# DataSet
### Source
- site: https://goldapple.ru/parfjumerija
- theme: **Парфюм**

Будем парсить только те товары, которые представляют из себя конкретный флакон духов/тулетной воды и т.п.

Исключаем из общего каталога все наборы, пробники, и всё, что не связано с туалетной водой (т.е. саше, мыло, гели и т.п.) и в случае их попадания в набор данных будем считать их ошибкой. [Ссылка на отфильтрованный каталог](https://goldapple.ru/parfjumerija?categories=1000000057,1000000059,1000142344,1000142345)
### Используемые инструменты
- requests
- BeautifulSoup
- selenium
- tqdm
- pandas
### Какие данные будем парсить
| Аттрибут | Формат    |
|----------|-----------|
| Название | Текст     |
| Артикул  | Текст |
| Тип (туал. вода, парф. вода и т.д.) | Категория |
| Цена (обыч.) | Число |
| Цена (макс. скидка) | Число |
| Описание | Текст |
| Объем | Число |
| Для кого (жен/муж/унисекс) | Категория |
| Группа ароматов | Tекст |
| Верхние ноты | Текст |
| Средние ноты | Текст |
| Базовые ноты | Текст |
| Бренд | Текст |
| Страна бренда | Текст |
| Страна происхождения | Текст |
| Число отзывов | Число |
| Оценка | Число |




## Парсинг

In [36]:
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

HOME_LINK = "https://goldapple.ru/"
CATALOG_LINK = "https://goldapple.ru/parfjumerija?categories=1000000057,1000000059,1000142344,1000142345"

### Получение HTML страницы каталога
Так как сайт использует динамическую подгрузку страниц с помощью JavaScript, будем листать до конца каталога с помощью *Selenium*

In [3]:
driver = webdriver.Chrome()
driver.get(CATALOG_LINK)

Код, выполняющий прокрутку сайта до конца:

In [ ]:
height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Листаем на всю высоту страницы (с небольшим отступом, чтобы было видно кнопку дозагрузки)
    driver.execute_script(f"window.scrollTo(0, {height - 800});")
    sleep(3)

    # Сравниваем, подгрузились ли новые страницы каталога (т.е. увеличилась ли высота сайта)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == height:
        try:
            load_button = driver.find_element(By.CSS_SELECTOR, 'button[data-transaction-name="ga-load-button"]')
            load_button.click()
        except Exception:
            print("Finished loading")
            break
    height = new_height

catalog_source_page = driver.page_source
driver.close()


In [10]:
with open('var/raw_catalog_page.txt', 'w', encoding='utf-8') as f:
    f.write(catalog_source_page)

Вся страница каталога лежит в **catalog_source_page**

### Получение списка ссылок на товары
С помощью *BeautifulSoup* строим дерево HTML

In [4]:
from bs4 import BeautifulSoup

with open('var/raw_catalog_page.txt', 'r', encoding='utf-8') as f:
    catalog_tree = BeautifulSoup(f, 'html.parser')

И получаем список ссылок на все отдельные товары в каталоге

In [ ]:
from tqdm import tqdm

item_list = []
for item in tqdm(catalog_tree.find_all('a', {'data-transaction-name': 'ga-product-card-vertical'})):
    item_list.append(HOME_LINK + item.attrs['href'])

print(len(item_list))

Сохраним список все ссылок в файл

In [ ]:
with open('var/link_list.txt', '', encoding='utf-8') as f:
    for link in tqdm(item_list):
        f.write(link + '\n')

### Парсинг отдельных страниц товара

Теперь нужно пройти по каждой ссылке из списка и получить необходимые данные с каждой страницы

In [4]:
import requests
from bs4 import BeautifulSoup


with open('var/link_list.txt', 'r', encoding='utf-8') as f:
    link_list = list(map(str.strip, f.readlines()))

Создадим специальный класс для парсинга

In [3]:
import pandas as pd
import re

def safe_get(func, default = None):
      try:
            return func()
      except Exception:
            return default

# Класс, который будет содержать в себе всю информацию, полученную с парсинга
class Parfume: 
      url: str = None
      name: str = None
      article_number: str = None
      parfume_type = None
      price_full_rub = None
      price_sale_rub = None
      description = None
      volume_ml = None
      for_whom = None
      aroma_group = None
      top_notes = None
      mid_notes = None
      base_notes = None
      brand = None
      brand_country = None
      produce_country = None
      ratings_count = None
      rating = None

      def __init__(self, url: str, soup: BeautifulSoup):
            self.url = url

            main = soup.find('main')

            # Из header получаем тип (parfume_type)
            self.parfume_type = safe_get(lambda: main.find('div', {'class': 'JMSge'}).text.strip())

            # Получаем цены (price_full_rub и price_sale_rub)
            prices = set()
            tmp = safe_get(lambda: main.find_all('div', {'class': 'kmEv4 B0Qrx'}))
            if (tmp != None):
                  for tag in tmp:
                        try:
                              prices.add(int(tag.text.strip().replace(' ', '')[:-1]))
                        except Exception:
                              pass
            
            tmp = safe_get(lambda: main.find_all('div', {'class': 'kmEv4 ybNWf dUtmC B0Qrx'}))
            if (tmp != None):
                  for tag in tmp:
                        try:
                              prices.add(int(tag.text.strip().replace(' ', '')[:-1]))
                        except Exception:
                              pass
            if (len(prices) > 0):
                  self.price_full_rub = max(prices)
                  self.price_sale_rub = min(prices)
            

            # Получаем данные из блока описания (название, артикул и описание)
            description = safe_get(lambda: main.find('div', {'value': 'Description_0'}))
            self.name = safe_get(lambda: description.find('div', {'class': 'OTJ7J'}).text.strip())
            self.article_number = safe_get(lambda: description.find('div', {'class': 'yh48W'}).text.strip())
            self.description = safe_get(lambda: description.find('div', {'class': 'tDMJt'}).text.strip().replace('\n', '').replace('\t', ''))
            
            # Данные из списка описания
            desc_list = safe_get(lambda: description.find_all('div', {'class': '_76A6k'}))
            if (desc_list != None):
                  for desc_item in desc_list:
                        desc_name = safe_get(lambda: desc_item.find('dt', {'class': '_9QHZV'}).text.strip())
                        desc_value = safe_get(lambda: desc_item.find('dt', {'class': 'Owxb0'}).text.strip())
                        if desc_name == 'для кого':
                              self.for_whom = desc_value
                        elif desc_name == 'группа ароматов':
                              self.aroma_group = desc_value
                        elif desc_name == 'верхние ноты':
                              self.top_notes = desc_value
                        elif desc_name == 'средние ноты':
                              self.mid_notes = desc_value
                        elif desc_name == 'базовые ноты':
                              self.base_notes = desc_value
                        elif desc_name == 'объём':
                              self.volume_ml = desc_value.split(' ')[0]

            # Информация о бренде
            brand = safe_get(lambda: main.find('div', {'value': re.compile('Brand_[0-9]{1}')}))
            self.brand = safe_get(lambda: brand.find('div', {'class': 'OTJ7J'}).text.strip())
            self.brand_country = safe_get(lambda: brand.find('div', {'class': 'yh48W'}).text.strip())

            # Информация о стране производства
            text = safe_get(lambda: main.find('div', {'text': 'Дополнительная информация'}).text.strip())
            if text != None:
                  start_index = len('страна происхождения')
                  end_index = text.find('изготовитель')
                  if (end_index == -1):
                        self.produce_country = safe_get(lambda: text[start_index:])
                  else:
                        self.produce_country = safe_get(lambda: text[start_index:end_index])

            # Информация об отзывах
            rating_responce = safe_get(lambda: requests.get(HOME_LINK + 'review/product/' + self.article_number))
            rating_tree = safe_get(lambda: BeautifulSoup(rating_responce.content, 'html.parser'))
            self.rating = safe_get(lambda: rating_tree.find('div', {'itemprop': 'ratingValue'}).text.strip())
            self.ratings_count = safe_get(lambda: rating_tree.find('meta', {'itemprop': 'reviewCount'}).attrs['content'].strip())

      def to_df(self):
            data = {
                  'url': self.url,
                  'title': self.name,
                  'article': self.article_number,
                  'parfume_type': self.parfume_type,
                  'full_price_rub': self.price_full_rub,
                  'sale_price_rub': self.price_sale_rub,
                  'volume_ml': self.volume_ml,
                  'for_whom': self.for_whom,
                  'aroma_group': self.aroma_group,
                  'top_notes': self.top_notes,
                  'mid_notes': self.mid_notes,
                  'base_notes': self.base_notes,
                  'description': self.description,
                  'rating': self.rating,
                  'ratings_count': self.ratings_count,
                  'brand': self.brand,
                  'brand_country': self.brand_country,
                  'produce_country': self.produce_country
            }
            df = pd.DataFrame(data, index=[0])
            return df

Теперь можно приступить к парсингу

In [2]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from time import sleep


Создаем *DataFrame*, в который будем помещать все данные

In [137]:
common_df = pd.DataFrame()

In [18]:
common_df = pd.read_csv('data/dataset.tsv', sep='\t', quoting=1, dtype=str)
# common_df.head()

In [7]:
timeouted = set()

Сама функция парсинга

In [13]:

n = len(common_df)
for i in tqdm(range(n, len(link_list))):
    link = link_list[i]
    sleep(0.75)

    try:
        response = requests.get(link, timeout=5)
    except Exception:
        timeouted.add(link)
        continue

    tree = BeautifulSoup(response.content, 'html.parser')
    parfume = Parfume(link, tree)

    if parfume.name == None:
        for j in range(6):
            sleep(0.75)
            try:
                response = requests.get(link, timeout=5)
            except Exception:
                timeouted.add(link)
                continue
            tree = BeautifulSoup(response.content, 'html.parser')
            parfume = Parfume(link, tree)
            if (parfume.name != None):
                break
    
    common_df = pd.concat([common_df, parfume.to_df()], ignore_index=True)
    common_df.to_csv('data/dataset.tsv', sep='\t', index=False, quoting=1)


100%|██████████| 7/7 [00:18<00:00,  2.70s/it]


Некоторые из запросов не выполнились из-за timeout'a

In [11]:
print(len(timeouted))

7


## Унифицирование

In [100]:
import pandas as pd
import re

### Удаление неудачных попыток
Удалим из набора объекты, которые не удалось спарсить, обыкновенно это объекты у которых все поля пустые, кроме может быть одного

In [142]:
dataset = pd.read_csv('data/dataset.tsv', sep='\t', quoting=1, dtype=str)
dataset = dataset[dataset.title.notnull()]

In [143]:
dataset.head(10)

,url,title,article,parfume_type,full_price_rub,sale_price_rub,volume_ml,for_whom,aroma_group,top_notes,mid_motes,base_notes,description,rating,ratings_count,brand,brand_country,produce_country
0,https://goldapple.ru//19000117827-aqua-millefolia,LE COUVENT Aqua Millefolia,19000117827,Туалетная вода,4900,3185,50,унисекс,"травяные, цитрусовые","лимон, мята","вербена, гальбанум",гваяковое дерево,Аромат AQUA MILLEFOLIA посвящен вавилонским Ви...,4.5,17,Le Couvent,Франция,Франция
1,https://goldapple.ru//26180600015-famille-roya...,12 PARFUMEURS FRANCAIS FAMILLE ROYALE Le Roi C...,26180600015,Духи,29700,25245,100,унисекс,"акватические, фруктовые","бергамот, зеленое яблоко, лаванда и арбуз",кедр и морские оттенки,"ваниль, пачули и ветивер",Духи 12 Parfumeurs Francais Famille Royale Le ...,0.0,0,12 PARFUMEURS FRANCAIS,Франция,Франция
2,https://goldapple.ru//19000048601-meadow-tea,NŌSE PERFUMES MEADOW TEA,19000048601,Парфюмерная вода,14900,11175,33,унисекс,пудровые,"мадагаскарская цитронелла, монарда, цветы липы","абсолют зеленого чая, почки тополя, ваниль","цистус, амбра","Мягкий и одновременно строгий аромат, где соче...",0.0,0,NŌSE perfumes,Россия,Россия
4,https://goldapple.ru//19000213214-musc-ravageu...,FREDERIC MALLE Musc Ravageur Holiday Limited e...,19000213214,Парфюмерная вода,29700,29700,100,унисекс,амбровые,бергамот,"корица, амбра","ваниль, мускус",Musc Ravageur был выпущен в 2000 году и обозна...,0.0,0,Frederic Malle,Франция,Франция
5,https://goldapple.ru//19000220504-gambit,MIND GAMES GAMBIT,19000220504,Парфюмерный экстракт,25550,22995,100,унисекс,"кожаные, древесные","петитгрейн, лаванда, гвоздика","кардамон, мадагаскарская герань, мимоза","пачули, сандаловое дерево, амбростар",Коллекция SOULMATE воплощает силу и единство п...,5.0,4,Mind Games,США,США
6,https://goldapple.ru//19000120337-shiro,AJMAL SHIRO,19000120337,Парфюмерная вода,11500,8050,90,мужской,акватические,"лимон, яблоко, герань","акватический аккорд, роза, кашмеран","кедр, амбра,мускус","“Аромат - это все, что нужно для путешествия в...",0.0,0,Ajmal,ОАЭ,ОАЭ
7,https://goldapple.ru//19000281747-scusami,FILIPPO SORCINELLI scusami,19000281747,Духи,19500,18525,100,унисекс,цветочные,"слива «мирабелла», лимон, бергамот, гелиотроп,...","черная смородина, иланг-иланг, фрезия, жидкий ...","амбра, амбретта, кремовый сандал, пачули, кедр...","_scusami_ - это форма забвения, состоящая из б...",0.0,0,Filippo Sorcinelli,Италия,Италия
8,https://goldapple.ru//19000209596-heaven-can-wait,FREDERIC MALLE Heaven Can Wait,19000209596,Парфюмерная вода (pre-pack),22600,22600,50,унисекс,древесные,"перец пименто, гвоздика, амбретта, семена моркови","ирис, ветивер, кашмеран, кедр","ваниль, персик, слива, мускус",Фредерик Маль и Жан-Клод Эллена работали над а...,4.0,1,Frederic Malle,Франция,Франция
9,https://goldapple.ru//26772000002-cafe-aoud,MANCERA Café Aoud,26772000002,Парфюмерная вода,15730,7865,60,унисекс,восточные,"персик, черная смородина, бергамот","амбра, цветочные ноты, кофе","белый мускус, сладкие ноты, древесные ноты","Ароматный гимн древней цивилизации Византии, с...",5.0,1,NaN,NaN,NaN
10,https://goldapple.ru//81151100001-tendre-rever...,MARINA DE BOURBON Tendre Reverence,81151100001,Парфюмерная вода,4910,2946,30,женский,"цветочные, цитрусовые","бергамот, красная смородина и персик","розовый пион, манголия и фиалка","мускус, сандал и ваниль","Утонченный, сладкий и изысканный Tendre Revere...",4.5,6,Marina de Bourbon,Франция,Франция


In [144]:
dataset.to_csv('data/dataset.tsv', sep='\t', index=False, quoting=1)

### Parfume type - тип товара

Унифицирование категориального типа - *parfume_type*

In [181]:
dataset = pd.read_csv('data/dataset.tsv', sep='\t', quoting=1, dtype=str)

Для начала приведем всё к нижнему регистру и избавимся от лишних пробелов

In [182]:
dataset['parfume_type'] = dataset['parfume_type'].str.lower().str.strip().str.replace('\xa0', ' ')

Теперь посмотрим на значения отранжированные по количеству встреч и унифицируем некоторые из значений (Например, с опечатками, лишними символами или перефразы)

In [201]:
dataset['parfume_type'].value_counts().head(30)

parfume_type
парфюмерная вода        3316
туалетная вода           939
духи                     568
набор                    214
автопарфюм               168
экстракт                  91
дымка                     64
масляные духи             64
рефил                     52
одеколон                  37
твердые духи              30
дезодорант                11
парфюмерная эссенция      11
душистая вода              9
Name: count, dtype: int64

Все, что связано с **экстрактами** - одна категория

In [184]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(экстракт).*', 'экстракт', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(экстаркт).*', 'экстракт', str(x)))

Все **наборы** также в одну категорию

In [185]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(набор|сет|комплект).*', 'набор', str(x)))

Все **одеколоны** в одну категорию

In [186]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(одеколон).*', 'одеколон', str(x)))

Дымки, вуали, мисты - **дымка**

In [187]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(дымка|вуаль|мист|освежающая вода|для волос).*', 'дымка', str(x)))

Унифицируем все автомобильные ароматизаторы - **автопарфюм**

In [188]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(автом|автод).*', 'автопарфюм', str(x)))

**Масляные духи**

In [200]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*масляные духи.*', 'масляные духи', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*духи масляные.*', 'масляные духи', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*масляной основе.*', 'масляные духи', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*масляный парфюм|масло.*', 'масляные духи', str(x)))

**парфюмерная эссенция**

In [190]:

dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*эфирная композиция|концентрированное масло.*', 'парфюмерная эссенция', str(x)))

**Рефилы**

In [191]:

dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*рефил(л)?|сменный блок.*', 'рефил', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*рефил.*', 'рефил', str(x)))

Все упоминания парфюмерной воды - **парфюмерная вода**

In [192]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(парфюмерная|парфюмированная|парфюмированая) (вода).*', 'парфюмерная вода', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(вода парфюмерная).*', 'парфюмерная вода', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(парю(ф)?мерная вода).*', 'парфюмерная вода', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(пафюмерная вода).*', 'парфюмерная вода', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(эликсир|ароматизатор).*', 'парфюмерная вода', str(x)))

**Душистая вода**

In [193]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*душистая.*', 'душистая вода', str(x)))

Дезодоранты

In [194]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*дезод.*', 'дезодорант', str(x)))

Аналогично с **туалетной водой**

In [195]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(туалетна(я)?) (вода).*', 'туалетная вода', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*(вода туалетная).*', 'туалетная вода', str(x)))

**Духи**

In [196]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*концентрированные духи|духи концентрированные*', 'духи', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*духи роликовые|духи-спрей*', 'духи', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*мужские духи *', 'духи', str(x)))

Удалим неинтересующие категории

In [197]:
dataset = dataset[~dataset.parfume_type.str.contains('лосьон')]
dataset = dataset[~dataset.parfume_type.str.contains('тела')]
dataset = dataset[~dataset.parfume_type.str.contains('гель')]
dataset = dataset[~dataset.parfume_type.str.contains('шкатул')]
dataset = dataset[~dataset.parfume_type.str.contains('воронк')]
dataset = dataset[~dataset.parfume_type.str.contains('клатч')]
dataset = dataset[~dataset.parfume_type.str.contains('футляр')]
dataset = dataset[~dataset.parfume_type.str.contains('с роликовым аппликатором')]

In [ ]:
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*масляные духи.*', 'масляные духи', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*духи масляные.*', 'масляные духи', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*масляной основе.*', 'масляные духи', str(x)))
dataset['parfume_type'] = dataset['parfume_type'].apply(lambda x: re.sub('.*масляный парфюм|масло.*', 'масляные духи', str(x)))

In [202]:
dataset.to_csv('data/dataset.tsv', sep='\t', index=False, quoting=1)

### For whom - для кого

Унифицирование категориального типа - *for_whom*

In [203]:
dataset = pd.read_csv('data/dataset.tsv', sep='\t', quoting=1, dtype=str)

In [204]:
dataset['for_whom'] = dataset['for_whom'].str.lower().str.strip()

In [208]:
dataset['for_whom'].value_counts().head(30)
# print(dataset['for_whom'].unique())


for_whom
унисекс          3203
женский          1605
мужской           739
для девочек        20
для мальчиков       2
Name: count, dtype: int64

Унифицируем значения

In [206]:
dataset['for_whom'] = dataset['for_whom'].apply(lambda x: re.sub('.*унисекс.*', 'унисекс', str(x)) if pd.notna(x) else x)
dataset['for_whom'] = dataset['for_whom'].apply(lambda x: re.sub('для девочек, для мальчиков', 'унисекс', str(x)) if pd.notna(x) else x)

In [207]:

dataset.to_csv('data/dataset.tsv', sep='\t', index=False, quoting=1)

### brand_country и produce_country

Унифицирование категориальных полей *brand_country* и *produce_country*

In [ ]:
dataset = pd.read_csv('data/dataset.tsv', sep='\t', quoting=1, dtype=str)

In [ ]:
dataset['brand_country'] = dataset['produce_country'].str.lower().str.strip()
dataset['produce_country'] = dataset['produce_country'].str.lower().str.strip()

In [ ]:
dataset['brand_country'].value_counts().head(30)
# print(dataset['brand_country'].unique())

In [ ]:
dataset['produce_country'] = dataset['produce_country'].apply(lambda x: re.sub('.*россия.*', 'россия', str(x)) if pd.notna(x) else x)
dataset['brand_country'] = dataset['brand_country'].apply(lambda x: re.sub('.*россия.*', 'россия', str(x)) if pd.notna(x) else x)

In [ ]:
dataset.to_csv('data/dataset.tsv', sep='\t', index=False, quoting=1)

## Преобразование в ARFF

In [221]:
import pandas as pd

In [222]:
dataset = pd.read_csv('data/dataset.tsv', sep='\t', quoting=1, dtype=str)

Создадим определение типов признаков

In [224]:
types = {
    'url': 'string',
    'title': 'string',
    'article': 'string',
    'parfume_type': 'nominal',
    'full_price_rub': 'numeric',
    'sale_price_rub': 'numeric',
    'volume_ml': 'numeric',
    'for_whom': 'nominal',
    'aroma_group': 'string',
    'top_notes': 'string',
    'mid_notes': 'string',
    'base_notes': 'string',
    'description': 'string',
    'rating': 'numeric',
    'ratings_count': 'numeric',
    'brand': 'string',
    'brand_country': 'nominal',
    'produce_country': 'nominal'
}

Код функции для сохранения данных в формате ARFF

In [225]:
def convert_to_arff(df, relation_name, types, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        # Записываем заголовок @relation
        f.write(f"@relation {relation_name}\n\n")
        
        # Записываем каждый столбец как атрибут
        for column in df.columns:
            if types[column] == 'numeric':
                f.write(f"@attribute {column} numeric\n")
            elif types[column] == 'nominal':
                unique_values = df[column].dropna().unique()
                unique_values_str = ','.join(map(str, unique_values))
                f.write(f"@attribute {column} {{{unique_values_str}}}\n")
            elif types[column] == 'string':
                f.write(f"@attribute {column} string\n")
        
        # Записываем данные
        f.write("\n@data\n")
        for _, row in df.iterrows():
            row['description'] = row['description'].replace(',', r'\,') # Экранируем запятые в поле 'description'
            row_str = ','.join(map(str, row))
            f.write(f"{row_str}\n")

In [226]:
convert_to_arff(dataset, 'parfumes', types, 'data/dataset.arff')

## Предобработка

Создать копию `dataset.tsv` под названием `preproc_dataset.tsv`

### Brand

Приведение признака *brand* в нижний регистр и удаление лишних пробелов 

In [227]:
dataset = pd.read_csv('data/preproc_dataset.tsv', sep='\t', quoting=1, dtype=str)

In [228]:
dataset['brand'] = dataset['brand'].str.lower().str.strip()

In [229]:
dataset.to_csv('data/preproc_dataset.tsv', sep='\t', index=False, quoting=1)

### Получение названий

Удаление лишней информации из признака *title*

У всех названий первая часть это название бренда, поэтому можно от неё избавиться и оставить только уникальную часть

In [230]:
dataset = pd.read_csv('data/preproc_dataset.tsv', sep='\t', quoting=1, dtype=str)

In [231]:
dataset['title'] = dataset['title'].str.lower().str.strip().str.replace('\xa0', ' ')

In [232]:
dataset.head(10)

,url,title,article,parfume_type,full_price_rub,sale_price_rub,volume_ml,for_whom,aroma_group,top_notes,mid_notes,base_notes,description,rating,ratings_count,brand,brand_country,produce_country
0,https://goldapple.ru//19000117827-aqua-millefolia,le couvent aqua millefolia,19000117827,туалетная вода,4900,3185,50,унисекс,"травяные, цитрусовые","лимон, мята","вербена, гальбанум",гваяковое дерево,Аромат AQUA MILLEFOLIA посвящен вавилонским Ви...,4.5,17,le couvent,франция,франция
1,https://goldapple.ru//26180600015-famille-roya...,12 parfumeurs francais famille royale le roi c...,26180600015,духи,29700,25245,100,унисекс,"акватические, фруктовые","бергамот, зеленое яблоко, лаванда и арбуз",кедр и морские оттенки,"ваниль, пачули и ветивер",Духи 12 Parfumeurs Francais Famille Royale Le ...,0.0,0,12 parfumeurs francais,франция,франция
2,https://goldapple.ru//19000048601-meadow-tea,nōse perfumes meadow tea,19000048601,парфюмерная вода,14900,11175,33,унисекс,пудровые,"мадагаскарская цитронелла, монарда, цветы липы","абсолют зеленого чая, почки тополя, ваниль","цистус, амбра","Мягкий и одновременно строгий аромат, где соче...",0.0,0,nōse perfumes,россия,россия
3,https://goldapple.ru//19000213214-musc-ravageu...,frederic malle musc ravageur holiday limited e...,19000213214,парфюмерная вода,29700,29700,100,унисекс,амбровые,бергамот,"корица, амбра","ваниль, мускус",Musc Ravageur был выпущен в 2000 году и обозна...,0.0,0,frederic malle,франция,франция
4,https://goldapple.ru//19000220504-gambit,mind games gambit,19000220504,экстракт,25550,22995,100,унисекс,"кожаные, древесные","петитгрейн, лаванда, гвоздика","кардамон, мадагаскарская герань, мимоза","пачули, сандаловое дерево, амбростар",Коллекция SOULMATE воплощает силу и единство п...,5.0,4,mind games,сша,сша
5,https://goldapple.ru//19000120337-shiro,ajmal shiro,19000120337,парфюмерная вода,11500,8050,90,мужской,акватические,"лимон, яблоко, герань","акватический аккорд, роза, кашмеран","кедр, амбра,мускус","“Аромат - это все, что нужно для путешествия в...",0.0,0,ajmal,оаэ,оаэ
6,https://goldapple.ru//19000281747-scusami,filippo sorcinelli scusami,19000281747,духи,19500,18525,100,унисекс,цветочные,"слива «мирабелла», лимон, бергамот, гелиотроп,...","черная смородина, иланг-иланг, фрезия, жидкий ...","амбра, амбретта, кремовый сандал, пачули, кедр...","_scusami_ - это форма забвения, состоящая из б...",0.0,0,filippo sorcinelli,италия,италия
7,https://goldapple.ru//19000209596-heaven-can-wait,frederic malle heaven can wait,19000209596,парфюмерная вода,22600,22600,50,унисекс,древесные,"перец пименто, гвоздика, амбретта, семена моркови","ирис, ветивер, кашмеран, кедр","ваниль, персик, слива, мускус",Фредерик Маль и Жан-Клод Эллена работали над а...,4.0,1,frederic malle,франция,франция
8,https://goldapple.ru//26772000002-cafe-aoud,mancera café aoud,26772000002,парфюмерная вода,15730,7865,60,унисекс,восточные,"персик, черная смородина, бергамот","амбра, цветочные ноты, кофе","белый мускус, сладкие ноты, древесные ноты","Ароматный гимн древней цивилизации Византии, с...",5.0,1,mancera,NaN,NaN
9,https://goldapple.ru//81151100001-tendre-rever...,marina de bourbon tendre reverence,81151100001,парфюмерная вода,4910,2946,30,женский,"цветочные, цитрусовые","бергамот, красная смородина и персик","розовый пион, манголия и фиалка","мускус, сандал и ваниль","Утонченный, сладкий и изысканный Tendre Revere...",4.5,6,marina de bourbon,франция,франция


Функция, которая обрежет название бренда из названия духов

In [233]:
def remove_producer(row):
    if pd.notna(row['brand']):
        return str(row['title']).removeprefix(row['brand']).strip()
    else:
        return row['title']
        

In [234]:
dataset['title'] = dataset.apply(lambda row: remove_producer(row), axis=1)

сохранение

In [235]:
dataset.to_csv('data/preproc_dataset.tsv', sep='\t', index=False, quoting=1)

### Обработка признаков с массивами: aroma_group, top_notes, mid_notes, base_notes

Приведение полей к единому виду, удобному для дальнейшей работы

In [236]:
import re

dataset = pd.read_csv('data/preproc_dataset.tsv', sep='\t', quoting=1, dtype=str)

Приведем их все к нижнему регистру и удалим запятые и иные разделители

In [237]:
dataset['aroma_group'] = dataset['aroma_group'].str.lower().str.strip()
dataset['top_notes'] = dataset['top_notes'].str.lower().str.strip()
dataset['mid_notes'] = dataset['mid_notes'].str.lower().str.strip()
dataset['base_notes'] = dataset['base_notes'].str.lower().str.strip()

Удалим запятые и другие разделители

In [238]:
def func(x):
    j = []
    for w in re.split(r'\s*,\s*|\s+и\s+', str(x)):
        j.append(w.strip()) 
    return ';'.join(j)

dataset['aroma_group'] = dataset['aroma_group'].apply(lambda x: func(x) if pd.notna(x) else x)
dataset['top_notes'] = dataset['top_notes'].apply(lambda x: func(x) if pd.notna(x) else x)
dataset['mid_notes'] = dataset['mid_notes'].apply(lambda x: func(x) if pd.notna(x) else x)
dataset['base_notes'] = dataset['base_notes'].apply(lambda x: func(x) if pd.notna(x) else x)

In [239]:
dataset.head()

,url,title,article,parfume_type,full_price_rub,sale_price_rub,volume_ml,for_whom,aroma_group,top_notes,mid_notes,base_notes,description,rating,ratings_count,brand,brand_country,produce_country
0,https://goldapple.ru//19000117827-aqua-millefolia,aqua millefolia,19000117827,туалетная вода,4900,3185,50,унисекс,травяные;цитрусовые,лимон;мята,вербена;гальбанум,гваяковое дерево,Аромат AQUA MILLEFOLIA посвящен вавилонским Ви...,4.5,17,le couvent,франция,франция
1,https://goldapple.ru//26180600015-famille-roya...,famille royale le roi chanceux,26180600015,духи,29700,25245,100,унисекс,акватические;фруктовые,бергамот;зеленое яблоко;лаванда;арбуз,кедр;морские оттенки,ваниль;пачули;ветивер,Духи 12 Parfumeurs Francais Famille Royale Le ...,0.0,0,12 parfumeurs francais,франция,франция
2,https://goldapple.ru//19000048601-meadow-tea,meadow tea,19000048601,парфюмерная вода,14900,11175,33,унисекс,пудровые,мадагаскарская цитронелла;монарда;цветы липы,абсолют зеленого чая;почки тополя;ваниль,цистус;амбра,"Мягкий и одновременно строгий аромат, где соче...",0.0,0,nōse perfumes,россия,россия
3,https://goldapple.ru//19000213214-musc-ravageu...,musc ravageur holiday limited edition,19000213214,парфюмерная вода,29700,29700,100,унисекс,амбровые,бергамот,корица;амбра,ваниль;мускус,Musc Ravageur был выпущен в 2000 году и обозна...,0.0,0,frederic malle,франция,франция
4,https://goldapple.ru//19000220504-gambit,gambit,19000220504,экстракт,25550,22995,100,унисекс,кожаные;древесные,петитгрейн;лаванда;гвоздика,кардамон;мадагаскарская герань;мимоза,пачули;сандаловое дерево;амбростар,Коллекция SOULMATE воплощает силу и единство п...,5.0,4,mind games,сша,сша


In [240]:
dataset.to_csv('data/preproc_dataset.tsv', sep='\t', index=False, quoting=1)

### Заполнение пропусков

In [241]:
import pandas as pd

In [242]:
dataset = pd.read_csv('data/preproc_dataset.tsv', sep='\t', quoting=1, dtype=str)

Проверим, в каком из признаков есть отсутствующие значения

In [ ]:
dataset[dataset['base_notes'].isna()].head(30)

Для всех пропущенных значений **volume_ml** заполним их модой по *parfume_type*

In [244]:
dataset['volume_ml'] = dataset.groupby('parfume_type')['volume_ml'].transform(lambda x: x.fillna(x.mode()[0]))

Для полей **brand_country** и **produce_country** заполним пропуски следущим образом: если известно одно из полей, сделаем второе таким же. Если неизвестны оба, то присвоим значение моды

In [ ]:
# Случай, когда известно одно из полей
dataset['brand_country'].fillna(dataset['produce_country'], inplace=True)
dataset['produce_country'].fillna(dataset['brand_country'], inplace=True)

# Случай, когда неизвестны оба из полей
dataset['brand_country'].fillna(dataset['brand_country'].mode()[0], inplace=True)
dataset['produce_country'].fillna(dataset['produce_country'].mode()[0], inplace=True)

#### Заполнение пропусков у текстовых полей

Все объекты, у которых пропущены значения полей **aroma_group**, **top_notes**, **mid_notes**, **base_notes** заполним значениями *'None'*

In [ ]:
dataset['aroma_group'].fillna('None', inplace=True)
dataset['top_notes'].fillna('None', inplace=True)
dataset['mid_notes'].fillna('None', inplace=True)
dataset['base_notes'].fillna('None', inplace=True)

Либо можно заполнить их значением предыдущего/следующего объекта

In [ ]:
dataset['aroma_group'].fillna(method='bfill', inplace=True)
dataset['top_notes'].fillna(method='bfill', inplace=True)
dataset['mid_notes'].fillna(method='bfill', inplace=True)
dataset['base_notes'].fillna(method='bfill', inplace=True)

Либо можно вообще удалить объекты с пропусками (так как их всего около 200, т.е. $\approx3.5\%$ от размера данных) 

In [ ]:
dataset = dataset.dropna(subset=['aroma_group', 'top_notes', 'mid_notes', 'base_notes'])

In [248]:
dataset.to_csv('data/preproc_dataset.tsv', sep='\t', index=False, quoting=1)

### Перевод нецелевых категориальных признаков в числовые значения

In [249]:
import pandas as pd

In [250]:
dataset = pd.read_csv('data/preproc_dataset.tsv', sep='\t', quoting=1, dtype=str)

Нецелевые категориальные признаки - **brand_country** и **produce_country**

Переведем их в числовые категории с помощью **One-Hot Encoding**

In [251]:
dataset = pd.get_dummies(dataset, columns=['brand_country', 'produce_country'])

In [252]:
dataset.to_csv('data/preproc_dataset.tsv', sep='\t', index=False, quoting=1)

### Перевод массивов в One-Hot Encoding

In [57]:
import pandas as pd

In [84]:
dataset = pd.read_csv('data/preproc_dataset.tsv', sep='\t', quoting=1, dtype=str)

Преваратим признаки со списками - **aroma_group**, **top_notes**, **mid_notes**, **base_notes** в набор отдельных признаков с помощью *One-Hot Encoding*

В признаке aroma_group всего 17 уникальных значений, поэтому будем брать их все при кодировании

In [71]:
m = set()
for row in dataset['aroma_group']:
    if (pd.isna(row)):
        continue
    for w in row.split(';'):
        m.add(w)

for group in m:
    dataset['aroma_group_{0}'.format(group)] = 0

In [73]:
# Функция для обновления one-hot кодирования
def update_aroma_groups(row):
    if pd.isna(row['aroma_group']):
        return row
    aromas = row['aroma_group'].split(';')
    for aroma in aromas:
        row[f'aroma_group_{aroma}'] = 1
    return row

# Применяем функцию ко всем строкам DataFrame
dataset = dataset.apply(update_aroma_groups, axis=1)

Для признаков **top_notes**, **mid_notes**, **base_notes** так как их количество очень велико возьмем топ 100 значений и сделаем для них *One-Hot Encoding*

In [ ]:
d = dict()
for row in dataset['base_notes']:
    if (pd.isna(row)):
        continue
    for w in row.split(';'):
        d[w] = d.setdefault(w, 0) + 1
i = 1
m = set()
for name, count in sorted(d.items(), key=lambda x: x[1], reverse=True):
    m.add(name.replace(' ', '_'))
    i += 1
    if i == 101:
        break
print(m)
len(m)

In [ ]:
for note in m:
    dataset['base_notes_{0}'.format(note)] = 0

# Функция для обновления one-hot кодирования
def update_aroma_groups(row):
    if pd.isna(row['base_notes']):
        return row
    aromas = row['base_notes'].split(';')
    for aroma in aromas:
        col_name = f'base_notes_{aroma.replace(' ', '_')}'
        if col_name in dataset.columns:
            row[col_name] = 1
    return row

# Применяем функцию ко всем строкам DataFrame
dataset = dataset.apply(update_aroma_groups, axis=1)

In [ ]:
dataset.head()

In [95]:
dataset.to_csv('data/preproc_dataset.tsv', sep='\t', index=False, quoting=1)

### Нормализация числовых значений

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('data/preproc_dataset.tsv', sep='\t', quoting=1, dtype=str)

Для полей `sale_price_rub`, `full_price_rub`, `volume_ml`, `ratings_count` используем Min-Max нормализацию

(Мб неуместно делать столбцы `sale_price_rub`, `full_price_rub` независимо друг от друга?)

In [ ]:
category = 'volume_ml' # Здесь менять признак

##### Min-Max нормализация

In [ ]:
def normalize(x, min, max):
    return (float(x) - min) / (max - min)

col = pd.to_numeric(dataset[category])
dataset[category] = dataset[category].apply(lambda x: normalize(x, col.min(), col.max()))

##### Z-масштабирование

In [ ]:
def normalize(x, mean, var):
    return (float(x) - mean) / (var ** 0.5)

col = pd.to_numeric(dataset[category])
dataset[category] = dataset[category].apply(lambda x: normalize(x, col.mean(), col.var()))

In [ ]:
dataset.to_csv('data/preproc_dataset.tsv', sep='\t', index=False, quoting=1)